# Partie principale   

Datasets à utiliser : Yago

## Statistiques de base

* Retourner la liste des 10 propriétés les plus fréquentes. La sortie doit être une liste de couples (prop, freq) triée de manière décroissante.


In [ ]:
case class Triple(sujet: String, prop: String, objet: String)

val yagoFile = "/tmp/BDLE/dataset/yagoFacts5M.txt"

defined class Triple
yagoFile: String = /tmp/BDLE/dataset/yagoFacts5M.txt


In [ ]:
val yago = sc.textFile(yagoFile).
  map(ligne => ligne.split("\\t")).coalesce(8).
  map(tab => Triple(tab(0), tab(1), tab(2))).toDS()

yago.persist
yago.count
yago.show(5)

+------------+----------------+--------------------+
|       sujet|            prop|               objet|
+------------+----------------+--------------------+
|    <!PAUS3>|     <hasGender>|              <male>|
|    <!PAUS3>|<hasMusicalRole>|<wordnet_bass_104...|
|    <!PAUS3>|   <isCitizenOf>|           <Ukraine>|
|    <!PAUS3>|     <wasBornIn>|      <Philadelphia>|
|<!T.O.O.H.!>|     <hasGender>|              <male>|
+------------+----------------+--------------------+
only showing top 5 rows

yago: org.apache.spark.sql.Dataset[Triple] = [sujet: string, prop: string ... 1 more field]


In [ ]:
val props = yago.groupBy("prop").count.sort($"count".desc).limit(10)

props.show()

+----------------+-------+
|            prop|  count|
+----------------+-------+
|<isAffiliatedTo>|1116512|
|      <playsFor>| 772092|
|   <isCitizenOf>| 731207|
|   <isLocatedIn>| 512925|
|     <hasGender>| 486528|
|     <wasBornIn>| 405252|
|       <actedIn>| 242436|
|        <diedIn>| 131001|
|   <hasWonPrize>| 115476|
| <graduatedFrom>| 112670|
+----------------+-------+

props: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [prop: string, count: bigint]


* Retourner la liste des 10 noeuds ayant le plus grand degré sortant. Rappel Le degré sortant d'un noeud n est le nombre de triplets où n est le sujet. La sortie doit être une liste de couples (sujet, degré) triée de manière décroissante.

In [ ]:
val noeudsDegres = yago.groupBy("sujet").count.sort($"count".desc).limit(10)
noeudsDegres.show()

+--------------------+-----+
|               sujet|count|
+--------------------+-----+
|       <Ilaiyaraaja>|  878|
|        <Prem_Nazir>|  471|
|     <United_States>|  456|
|   <Deva_(composer)>|  440|
|       <Adoor_Bhasi>|  383|
| <M._S._Viswanathan>|  378|
|         <Mammootty>|  360|
|    <United_Kingdom>|  345|
|<Laxmikant–Pyarelal>|  324|
|          <Mohanlal>|  314|
+--------------------+-----+

noeudsDegres: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [sujet: string, count: bigint]


* Pour chaque propriété, retourner le nombre de sujets distincts d'où elle démarre ainsi que le nombre d'objets distincts où elle arrive. La sortie doit être une liste de tuples (pro, nb-sujets, nb-objets). Attention Un objet (sujet) peut avoir plusieurs fois la même propriété.

In [ ]:
val propSujets = yago.groupBy("prop", "sujet").count.groupBy("prop").count.withColumnRenamed("count", "nbSujet")
propSujets.show(5)

val propObjets = yago.groupBy("prop", "objet").count.groupBy("prop").count.withColumnRenamed("count", "nbObjet")
propObjets.show(5)

val propStats = propSujets.join(propObjets, "prop")
propStats.show(5)

+--------------------+-------+
|                prop|nbSujet|
+--------------------+-------+
|<hasOfficialLangu...|   1177|
|    <isInterestedIn>|    781|
|         <dealsWith>|    262|
|       <isLocatedIn>| 114649|
|<hasAcademicAdvisor>|   2318|
+--------------------+-------+
only showing top 5 rows

+--------------------+-------+
|                prop|nbObjet|
+--------------------+-------+
|<hasOfficialLangu...|    134|
|    <isInterestedIn>|    160|
|         <dealsWith>|    204|
|       <isLocatedIn>|  30544|
|<hasAcademicAdvisor>|    527|
+--------------------+-------+
only showing top 5 rows

+--------------------+-------+-------+
|                prop|nbSujet|nbObjet|
+--------------------+-------+-------+
|<hasOfficialLangu...|   1177|    134|
|    <isInterestedIn>|    781|    160|
|         <dealsWith>|    262|    204|
|       <isLocatedIn>| 114649|  30544|
|<hasAcademicAdvisor>|   2318|    527|
+--------------------+-------+-------+
only showing top 5 rows

propSujets: org.

* Encoder la fonction noeudDegre(d:entier) qui retourne les noeuds de degrée d. Le degré d'un noeud = degré sortant + degré entrant.

In [ ]:

def noeudDegre(d: Int) : org.apache.spark.sql.Dataset[org.apache.spark.sql.Row]  = {
  val noeudsDegres = yago.groupBy("sujet").count.sort($"count".desc).limit(10)
  noeudsDegres.where("count = "+d.toString)
}

noeudDegre(112670).show(5)


+-----+-----+
|sujet|count|
+-----+-----+
+-----+-----+

noeudDegre: (d: Int)org.apache.spark.sql.Dataset[org.apache.spark.sql.Row]
